# Une méthode pour filtrer les données

### Problématique

Les zones d'autorisation sont représentées par des points placés à certains endroits. Ces points sont plus ou moins espacés et semblent remplir l"espace pour former des zones. Les coordonnées des stations météo et des parcs éoliens existants sont également représentées par des points.

Pour filtrer et comparer ces données, il n"est pas possible de simplement soustraire les coordonnées, car la probabilité qu'une station météo se trouve exactement au même endroit qu'un point d'une zone est presque nulle.

Pour résoudre cette problématique, nous avons d'abord essayé la méthode du `ray casting`, qui consiste à projeter une droite dans toutes les directions autour d'un point jusqu'à rencontrer un autre point. Cette méthode est efficace pour les points situés sur le périmètre d'un polygone, mais moins adaptée lorsque les points remplissent l'aire de celui-ci. Nous avons également tenté d'utiliser la méthode `k-means` pour réduire la taille des zones à tester, mais les résultats n'ont pas été concluants.

Nous avons donc décidé de repartir de zéro en reformulant la question : **À quelle distance d'une zone se trouve une station météo ?**

Cette approche introduit immédiatement la notion de rayon autour d'un point et élimine la nécessité de superposer les points.

Après quelques recherches, nous avons découvert la formule de `Haversine`. Il s'agit d'une formule utilisée pour calculer la distance entre deux points sur une sphère à partir de leurs coordonnées de longitude et de latitude. En termes plus simples, elle permet de calculer la distance la plus courte entre deux points à la surface d'un objet sphérique, comme la Terre. Il suffisait ensuite de préciser une distance maximale autorisée, et le problème était résolu.

In [13]:
import random

import pandas as pd
import numpy as np
import plotly.express as px

In [14]:
R = 6371  # Earth Radius in kilometers
MAX_DIST = 30 # kilometers

In [15]:
df_zones = pd.read_csv("s3://jedha-final-project-jrat/zones.csv")
df_zones.head()

,Y,X,Latitude,Longitude,Color
0,79,571,50.198832,3.355932,"rgb(206,136,255)"
1,80,571,50.188113,3.355932,"rgb(206,136,255)"
2,80,579,50.188113,3.480975,"rgb(208,142,255)"
3,81,572,50.177394,3.371562,"rgb(206,136,255)"
4,83,577,50.155957,3.449714,"rgb(208,142,255)"


In [8]:
df_positions = pd.DataFrame({
    "Latitude": [random.uniform(43, 50) for _ in range(200)],
    "Longitude": [random.uniform(-1, 10) for _ in range(200)]
})
df_positions.head()  # remplacer par les vraies positions des stations meteo ou parcs eoliens

,Latitude,Longitude
0,48.132028,6.578735
1,44.308478,1.334661
2,49.255882,4.396361
3,46.773663,9.434161
4,46.117363,9.247404


In [19]:
def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return c * R


def check_authorization(df_zones: pd.DataFrame, df_positions: pd.DataFrame):
    authorized_positions = []

    for _, pos in df_positions.iterrows():
        pos_lat, pos_lon = pos["LAT"], pos["LON"]
        is_authorized = False
        
        for _, zone in df_zones.iterrows():
            zone_lat, zone_lon, zone_radius = zone["Latitude"], zone["Longitude"], zone['Radius']
            distance = haversine(pos_lat, pos_lon, zone_lat, zone_lon)
            
            if distance <= zone_radius:
                is_authorized = True
                break

        authorized_positions.append(is_authorized)

    df_positions["is_authorized"] = authorized_positions
    return df_positions


df_zones["Radius"] = MAX_DIST
df_positions_filtered = check_authorization(df_zones, df_positions)

df_positions_filtered.head()


C:\Users\giand\AppData\Local\Temp\ipykernel_1160\4143877806.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positions["is_authorized"] = authorized_positions


,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,...,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region,is_authorized
0,1014002,ARBENT,46.278167,5.669000,534,2004-03-01,2004,3,23.8,12.7,...,50.4,300.0,6.0,25.2,360.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True
226,1033002,BELLEGARDE,46.086500,5.814167,350,1994-04-01,1994,4,144.4,12.7,...,57.6,220.0,12.0,28.8,50.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True
570,1034004,BELLEY,45.769333,5.688000,330,2001-10-01,2001,10,98.4,20.8,...,90.0,200.0,5.0,50.4,200.0,1.0,1,Ain,AUVERGNE RHONE ALPES,False
825,1071001,CESSY,46.310333,6.080333,507,2002-05-01,2002,5,145.8,18.6,...,61.2,340.0,7.0,36.0,340.0,1.0,1,Ain,AUVERGNE RHONE ALPES,True
1073,1072001,CEYZERIAT_SAPC,46.204333,5.287667,260,1994-08-01,1994,8,36.0,26.9,...,43.2,220.0,4.0,21.6,190.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True


In [10]:
df_positions["is_authorized"] = df_positions["is_authorized"].astype(str)

# zones
fig = px.scatter_mapbox(
    df_zones, 
    lat="Latitude", 
    lon="Longitude",
    zoom=5,
    height=800
)

fig.update_traces(marker=dict(size=MAX_DIST, color="gray"))

# stations or parks
fig.add_scattermapbox(
    lat=df_positions["LAT"], 
    lon=df_positions["LON"], 
    mode="markers",
    marker=dict(size=10),
    marker_color=df_positions["is_authorized"].map({"True": "green", "False": "red"})
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()


![](https://raw.githubusercontent.com/tristanGIANDO/jedha_final_project/develop/jedha_final_project/src/filter_haversine.png)

---

# Filtering Meteo stations

In [4]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_03.csv")

In [11]:
df.shape

(340387, 26)

In [21]:
df_positions = df.drop_duplicates(subset=['LAT', 'LON'])
df_positions.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,...,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
0,1014002,ARBENT,46.278167,5.669000,534,2004-03-01,2004,3,23.8,12.7,...,7.6,50.4,300.0,6.0,25.2,360.0,0.0,1,Ain,AUVERGNE RHONE ALPES
226,1033002,BELLEGARDE,46.086500,5.814167,350,1994-04-01,1994,4,144.4,12.7,...,5.8,57.6,220.0,12.0,28.8,50.0,0.0,1,Ain,AUVERGNE RHONE ALPES
570,1034004,BELLEY,45.769333,5.688000,330,2001-10-01,2001,10,98.4,20.8,...,5.8,90.0,200.0,5.0,50.4,200.0,1.0,1,Ain,AUVERGNE RHONE ALPES
825,1071001,CESSY,46.310333,6.080333,507,2002-05-01,2002,5,145.8,18.6,...,6.8,61.2,340.0,7.0,36.0,340.0,1.0,1,Ain,AUVERGNE RHONE ALPES
1073,1072001,CEYZERIAT_SAPC,46.204333,5.287667,260,1994-08-01,1994,8,36.0,26.9,...,5.4,43.2,220.0,4.0,21.6,190.0,0.0,1,Ain,AUVERGNE RHONE ALPES


In [23]:
df_positions.shape

(1126, 26)

In [25]:
df_positions["is_authorized"] = False

C:\Users\giand\AppData\Local\Temp\ipykernel_1160\2663664555.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
df_zones["Radius"] = MAX_DIST
df_positions_filtered = check_authorization(df_zones, df_positions)

df_positions_filtered.head()

C:\Users\giand\AppData\Local\Temp\ipykernel_1160\4143877806.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,...,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region,is_authorized
0,1014002,ARBENT,46.278167,5.669000,534,2004-03-01,2004,3,23.8,12.7,...,50.4,300.0,6.0,25.2,360.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True
226,1033002,BELLEGARDE,46.086500,5.814167,350,1994-04-01,1994,4,144.4,12.7,...,57.6,220.0,12.0,28.8,50.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True
570,1034004,BELLEY,45.769333,5.688000,330,2001-10-01,2001,10,98.4,20.8,...,90.0,200.0,5.0,50.4,200.0,1.0,1,Ain,AUVERGNE RHONE ALPES,False
825,1071001,CESSY,46.310333,6.080333,507,2002-05-01,2002,5,145.8,18.6,...,61.2,340.0,7.0,36.0,340.0,1.0,1,Ain,AUVERGNE RHONE ALPES,True
1073,1072001,CEYZERIAT_SAPC,46.204333,5.287667,260,1994-08-01,1994,8,36.0,26.9,...,43.2,220.0,4.0,21.6,190.0,0.0,1,Ain,AUVERGNE RHONE ALPES,True


In [28]:
df_positions["is_authorized"] = df_positions["is_authorized"].astype(str)

# zones
fig = px.scatter_mapbox(
    df_zones, 
    lat="Latitude", 
    lon="Longitude",
    zoom=5,
    height=800
)

fig.update_traces(marker=dict(size=MAX_DIST, color="gray"))

# stations or parks
fig.add_scattermapbox(
    lat=df_positions["LAT"], 
    lon=df_positions["LON"], 
    mode="markers",
    marker=dict(size=10),
    marker_color=df_positions["is_authorized"].map({"True": "green", "False": "red"})
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

C:\Users\giand\AppData\Local\Temp\ipykernel_1160\1800820054.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

